In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPUs to use

In [2]:
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, 
)

from peft import get_peft_model, LoraConfig, TaskType, PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Seoul')}
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:

In [3]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(1, 64)

In [4]:
torch.cuda.is_available()

True

In [5]:
MODEL_CHECKPOINT = 'EleutherAI/gpt-j-6b'
MODEL_CHECKPOINT = 'ainize/gpt-j-6B-float16'
MODEL_CHECKPOINT = 'ainize/kobart-news'
MODEL_CHECKPOINT = 'paust/pko-t5-base'
MODEL_CHECKPOINT = 'EleutherAI/polyglot-ko-5.8b'
MODEL_CHECKPOINT = 'EleutherAI/polyglot-ko-1.3b'

In [6]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [7]:
config.architectures[0].lower()

'gptneoxforcausallm'

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_CHECKPOINT,
    config=config,
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [32]:
source = ['화이팅']
source = tokenizer(source, max_length=2048, truncation=True, return_tensors='pt')

In [33]:
source = {k:v for k, v in source.items() if k != 'token_type_ids'}

In [45]:
model.eval()
with torch.no_grad(), torch.inference_mode():
    outputs = model.generate(**source, max_new_tokens=32, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

In [46]:
tokenizer.batch_decode(outputs)

['화이팅!!!<|endoftext|>']

In [43]:
tokenizer.pad_token_id, tokenizer.eos_token_id

(2, 2)

In [37]:
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "transformers_version": "4.28.1"
}